# 冒烟测试

快速验证 holdem-lab 已正确安装并能正常工作。

In [ ]:
# 导入所有模块
from holdem_lab import (
    Card, Deck, Rank, Suit,
    parse_card, parse_cards, format_cards,
    HandType, HandRank, evaluate_hand, find_winners,
    EquityRequest, PlayerHand, calculate_equity,
    EventLog, EventType, HandReplayer,
    GameState, Street,
)
print("所有模块导入成功！")

## 1. 卡牌解析

In [ ]:
# 解析卡牌
cards = parse_cards("Ah Kh Qh Jh Th")
print(f"解析结果: {format_cards(cards)}")
print(f"花色显示: {' '.join(c.pretty() for c in cards)}")

## 2. 手牌评估

In [ ]:
# 评估不同手牌
test_hands = [
    ("Ah Kh Qh Jh Th", "Royal Flush"),
    ("9h 8h 7h 6h 5h", "Straight Flush"),
    ("Ah Ad Ac As Kh", "Four of a Kind"),
    ("Ah Ad Ac Kh Kd", "Full House"),
    ("Ah Kh Qh Jh 9h", "Flush"),
    ("Ah Kd Qc Js Th", "Straight"),
    ("Ah Ad Ac Kh Qd", "Three of a Kind"),
    ("Ah Ad Kh Kd Qc", "Two Pair"),
    ("Ah Ad Kh Qd Jc", "One Pair"),
    ("Ah Kd Qc Js 9h", "High Card"),
]

print("手牌评估测试:")
print("-" * 50)
for cards_str, expected in test_hands:
    cards = parse_cards(cards_str)
    rank = evaluate_hand(cards)
    status = "✓" if expected in str(rank.hand_type) else "✗"
    print(f"{status} {cards_str:25} -> {rank.describe()}")

## 3. 权益计算

In [ ]:
# 经典对局：AA vs KK
request = EquityRequest(
    players=[
        PlayerHand(hole_cards=tuple(parse_cards("Ah Ad"))),
        PlayerHand(hole_cards=tuple(parse_cards("Kh Kd"))),
    ],
    num_simulations=10000,
    seed=42,
)

result = calculate_equity(request)
print("AA vs KK:")
print(f"  AA 权益: {result.players[0].equity:.1%}")
print(f"  KK 权益: {result.players[1].equity:.1%}")
print(f"  (预期: ~82% vs ~18%)")

## 4. 游戏状态

In [ ]:
# 运行完整牌局
game = GameState(num_players=3, seed=42)
result = game.run_to_showdown()

print("牌局结果:")
print(f"  公共牌: {format_cards(game.board)}")
for i, player in enumerate(game.players):
    print(f"  玩家 {i}: {format_cards(player.hole_cards)}")
print(f"  获胜者: 玩家 {result.winners[0]}")
print(f"  获胜牌型: {result.winning_hand.describe()}")

## 5. 事件日志

In [ ]:
# 查看事件日志
log = game.event_log
print(f"事件总数: {len(log)}")
print("\n事件类型:")
for event in log:
    print(f"  {event.event_type.value}")

## 所有测试通过！

如果你在上方看到了对勾 (✓) 和合理的输出，说明 holdem-lab 工作正常。